In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np
#import cupy as cp
from tqdm import tqdm
tqdm.pandas()

In [3]:
class CFG:
    exp_no = "contents-for-retriever_v4" # 毎回変える＆確認する事！
    # Load Paths: Common(基本変えなくていい)
    # competition_data
    compdata_topics = "/content/drive/MyDrive/Colab_Files/kaggle/lecr/input/00_competition/topics.csv"
    compdata_content = "/content/drive/MyDrive/Colab_Files/kaggle/lecr/input/00_competition/content.csv"
    compdata_correlations = "/content/drive/MyDrive/Colab_Files/kaggle/lecr/input/00_competition/correlations.csv"
    compdata_sample_sub = "/content/drive/MyDrive/Colab_Files/kaggle/lecr/input/00_competition/sample_submission.csv"

    # Save paths（動的に変わるので基本いじらなくていい）
    # data save path
    data_save_path = f"/content/drive/MyDrive/Colab_Files/kaggle/lecr/input/01_for_retriever/{exp_no}"

print("data_path: ", CFG.data_save_path)
! mkdir -p {CFG.data_save_path}

data_path:  /content/drive/MyDrive/Colab_Files/kaggle/lecr/input/01_for_retriever/contents-for-retriever_v4


In [4]:
# =========================================================================================
# Utils
# =========================================================================================
import pickle

def show_df(df:pd.DataFrame, rows_=5):
    print(df.shape)
    return display(df.head(rows_))

def pickle_dump(obj_, filename):
    with open(f'{filename}', 'wb') as f:
        pickle.dump(obj_, f)
        
def pickle_load(path_):
    with open(f'{path_}', 'rb') as f:
        obj_ = pickle.load(f)
    return obj_

def create_list_batch(list_:list, batch_size:int):
    out_ = []
    for i in range(len(list_)):
        batch_ = list_[i*batch_size:(i+1)*batch_size]
        if len(batch_) == 0:
            pass
        else:
            out_.append(batch_)
    return out_


# =========================================================================================
# Token追加関連
# =========================================================================================
# channelからtoken文字列を生成する関数
def create_token_str_dic(df:pd.DataFrame(), column=str):
    token_str_dic = {i:f"[{column}={i}]" for i in df[column].unique()}
    return token_str_dic

# token文字列をtokenizerに追加
def add_token_str_to_tokenizer(token_str_dic, null_token, cfg):
    print(len(cfg.tokenizer))
    tokens = [i[1] for i in token_str_dic.items()] + [null_token]
    cfg.tokenizer.add_tokens(tokens)
    print(len(cfg.tokenizer))
    lengths = []
    for text in tqdm(tokens, total = len(tokens)):
        length = len(cfg.tokenizer(text, add_special_tokens = False)['input_ids'])
        lengths.append(length)
    margin_length = max(lengths)
    return margin_length

# token文字列をdfに適用する処理(辞書に無いchannelはothersとする)
def convert_texts_to_token_str(sr:pd.Series(), token_str_dic, null_token):    
    out_sr = sr.map(token_str_dic)
    out_sr = out_sr.fillna(null_token)
    return out_sr

# topicsのdfとカラム名を与えると、処理後のtopicsとtoken辞書を返す処理
def category_cols_to_token_str_col(topics, column):
    token_str_dic = create_token_str_dic(topics, column)
    null_token = f"[{column}==others]"
    topics[f'{column}_token'] = convert_texts_to_token_str(topics[column], token_str_dic, null_token)
    return topics, token_str_dic


<ipython-input-4-db71bfc3b7c3>:52: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  def convert_texts_to_token_str(sr:pd.Series(), token_str_dic, null_token):


In [5]:
DEBUG = False

# Load data
print('Load data')
content = pd.read_csv(CFG.compdata_content)
correlations = pd.read_csv(CFG.compdata_correlations)
if DEBUG:
    topics = topics.sample(500)

# fillna
print('fillna')
content['title'] = content['title'].fillna("")
content['description'] = content['description'].fillna("")
content['text'] = content['text'].fillna("")

# cut length
print('cut length')
content['title'] = content['title'].map(lambda x: ' '.join(x.split(' ')[:46]))
content['description'] = content['description'].map(lambda x: ' '.join(x.split(' ')[:98]))

# token文字列への変換＆辞書の取得
print('convert categories to token str')
content_token_str_dics = {}
content, content_token_str_dics['kind'] = category_cols_to_token_str_col(content, 'kind')
content, content_token_str_dics['language'] = category_cols_to_token_str_col(content, 'language')
#content, content_token_str_dics['copyright_holder'] = category_cols_to_token_str_col(content, 'copyright_holder')
#content, content_token_str_dics['license'] = category_cols_to_token_str_col(content, 'license')

# concat text
print('concat text')
content['all_text'] = content['kind_token'] + '[SEP]' \
                     + content['language_token'] + '[SEP]' \
                     + content['title'] + '[SEP]' \
                     + content['description'] + '[SEP]' \
                     + content['text']

Load data
fillna
cut length
convert categories to token str
concat text


In [6]:
pickle_dump(content, f'{CFG.data_save_path}/content_preprocessed_for_retriever.pkl')
pickle_dump(content_token_str_dics, f'{CFG.data_save_path}/content_token_str_dics.pkl')

In [7]:
content_load = pickle_load(f'{CFG.data_save_path}/content_preprocessed_for_retriever.pkl')

In [8]:
content_load.head(3)

,id,title,description,kind,text,language,copyright_holder,license,kind_token,language_token,all_text
0,c_00002381196d,"Sumar números de varios dígitos: 48,029+233,930","Suma 48,029+233,930 mediante el algoritmo está...",video,,es,NaN,NaN,[kind=video],[language=es],[kind=video][SEP][language=es][SEP]Sumar númer...
1,c_000087304a9e,Trovare i fattori di un numero,Sal trova i fattori di 120.\n\n,video,,it,NaN,NaN,[kind=video],[language=it],[kind=video][SEP][language=it][SEP]Trovare i f...
2,c_0000ad142ddb,Sumar curvas de demanda,Cómo añadir curvas de demanda\n\n,video,,es,NaN,NaN,[kind=video],[language=es],[kind=video][SEP][language=es][SEP]Sumar curva...
